<h1>2. Boundary Nodes</h1>
<blockquote>
    This notebook is divided into 2 sections.
    Section 1 uses the Overpass API to obtain the nodes of the boundary and save them to a .osm (xml) file.
    Section 2 saves the lat, lon of each node in the boundary in a csv file.
</blockquote>

In [1]:
import csv
import requests
import os
import osmium as o
import xml.etree.ElementTree as ET
import xmltodict

>Enter the OSM Relation ID of the boundary below. 

In [2]:
# User Overpass API to get the nodes of an osm boundary given the boundary's Relation ID
boundary_rel_id = 4136816  # OSM Relation ID of boundary
initial_boundary_file = f'{boundary_rel_id}_boundary_nodes.osm'  # nodes only
boundary_nodes_lat_lon = f'{boundary_rel_id}_boundary_nodes.csv'
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = f'[out:xml];(rel(id:{boundary_rel_id})->.a;way(r.a)->.b;node(w.b)->.c;);.c out geom;'

<h2>Section 1. Boundary Nodes in XML Using Overpass API</h2>
<blockquote>
    Only the nodes of the boundary are saved in the .osm (xml) file.
</blockquote>

In [5]:
response = requests.get(overpass_url, params={'data': overpass_query})
tree = ET.fromstring(response.content)
with open(initial_boundary_file, "wb") as f:
    f.write(ET.tostring(tree))

<h2>Section 2. Lat, Lon of Boundary Nodes</h2>
<blockquote>
    Output is a csv containing lat, lon of each node in the boundary.
</blockquote>

In [6]:
tree = ET.parse(initial_boundary_file)
xml_data = tree.getroot()
xmlstr = ET.tostring(xml_data, encoding='utf8', method='xml')
data_dict = dict(xmltodict.parse(xmlstr))['osm']['node']
list_data_dict = [{'lat':float(r['@lat']), 'lon':float(r['@lon'])} for r in data_dict]
keys = list_data_dict[0].keys()
with open(boundary_nodes_lat_lon, 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys, extrasaction='ignore')
    dict_writer.writeheader()
    dict_writer.writerows(list_data_dict)